# Import the Necessary Libraries

In [1]:
import torch 
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchsummary import summary # pip install torch-summary
from tqdm import tqdm 

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cpu


In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Download the CIFAR 10 Dataset

In [4]:
# Get the training data and create a transform to ensure the data is made up of tensors
# why are we not normalizing the data between 0 and 1?
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform=transform)

# load the training data and transform it
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

# Get the test data  
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform=transform)

# Now load the test data with a batch size of 32: Do we need to shuffle the test data? 
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2) #None # write your code here

Files already downloaded and verified
Files already downloaded and verified


# Building the CNN Model

In [19]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # Create instance variables for the layers you will pass in the forward method
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        
        # Define the fully connected (linear) layers
        self.fc1 = torch.nn.Linear(32 * 8 * 8, 256)  # Assuming input image size is 32x32
        self.fc2 = torch.nn.Linear(256, 10)  # Output size is 10 classes
        
    def forward(self, x):
        # Apply first convolutional layer followed by ReLU activation and max pooling
        x = torch.nn.functional.max_pool2d(torch.nn.functional.relu(self.conv1(x)), 2)
        # Apply second convolutional layer followed by ReLU activation and max pooling
        x = torch.nn.functional.max_pool2d(torch.nn.functional.relu(self.conv2(x)), 2)
        
        # Reshape the tensor for fully connected layers
        x = x.view(-1, 32 * 8 * 8)  # Flatten the tensor
        
        # Apply fully connected layers with ReLU activation
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Instantiate the model
cnn_model = CNN()

# summary(cnn_model)
print(cnn_model)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [25]:
import torch

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Create instance variables for the layers you will pass in the forward method
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
#         self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=1, padding=1)
#         self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1)
#         self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
#         self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=1)
        
        
        # Define the fully connected (linear) layers
        self.fc1 = torch.nn.Linear(64 * 8 * 8, 128)  # Input size based on output size of last convolutional layer
        self.dropout = torch.nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(128, 10)  # Output size is 10 classes

    def forward(self, x):
        # Apply first convolutional layer followed by batch normalization and leaky ReLU activation
        x = torch.nn.functional.leaky_relu(torch.nn.BatchNorm2d(self.conv1(x)), negative_slope=0.01)
        # Apply second convolutional layer followed by batch normalization and leaky ReLU activation
        x = torch.nn.functional.leaky_relu(torch.nn.BatchNorm2d(self.conv2(x)), negative_slope=0.01)
        # Apply third convolutional layer followed by batch normalization and leaky ReLU activation
        x = torch.nn.functional.leaky_relu(torch.nn.BatchNorm2d(self.conv3(x)), negative_slope=0.01)
        # Apply fourth convolutional layer followed by batch normalization and leaky ReLU activation
        x = torch.nn.functional.leaky_relu(torch.nn.BatchNorm2d(self.conv4(x)), negative_slope=0.01)
        # Reshape the tensor for fully connected layers
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor
        # Apply fully connected layers with ReLU activation
        x = torch.nn.functional.leaky_relu(torch.nn.BatchNorm1d(self.fc1(x)), negative_slope=0.01)
        x = self.dropout(x)
        # Apply the final fully connected layer without activation function
        x = self.fc2(x)
        return x

# Instantiate the model
cnn_model = CNN()

print(cnn_model)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [59]:
import math

# Calculate the output size of a convolutional layer with "same" padding
def calculate_output_size(input_size, kernel_size, stride, padding):
    return math.floor((input_size + 2 * padding - kernel_size) / stride) + 1

# Function to calculate the padding required to maintain the input size
def calculate_same_padding(input_size, kernel_size, stride):
    # Formula to calculate the padding
    padding = ((input_size - 1) * stride + kernel_size - input_size) // 2
    return padding

class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # Create instance variables for the layers you will pass in the forward method
        # write your code below and ensure your padding is correct and well calculated
        import math
        
        # calculating the padding
        def calculate_padding(input_size, kernel_size, stride, output_size):
            padding = ((output_size - 1) * stride - input_size + kernel_size) / 2
            return math.floor(padding)
        
        padding_number = 1
        
        self.convLayer1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=1, padding=padding_number)
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.convLayer2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=padding_number)
        self.bn2 = torch.nn.BatchNorm2d(32)
        
        self.convLayer3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=padding_number)
        self.bn3 = torch.nn.BatchNorm2d(64)
        
        self.convLayer4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=padding_number)
        self.bn4 = torch.nn.BatchNorm2d(64)
        
        # Calculate the output size of the last convolutional layer
        last_conv_output_size = calculate_output_size(32, 4, 1, padding_number)  # Assuming input size 32x32
        last_conv_output_size = calculate_output_size(last_conv_output_size, 3, 2, padding_number)
        last_conv_output_size = calculate_output_size(last_conv_output_size, 3, 1, padding_number)
        last_conv_output_size = calculate_output_size(last_conv_output_size, 3, 2, padding_number)
        
        
        self.fclayer1 = torch.nn.Linear(64 * last_conv_output_size * last_conv_output_size, 128) # This dimension depends on the input image size and conv layers 32 * 8 * 8 
        self.bn5 = torch.nn.BatchNorm1d(128)
        
        self.fclayer2 = torch.nn.Linear(128 , 10) 
        
        
    def forward(self, x):
        # complete your forward method
        x=torch.nn.functional.leaky_relu(self.bn1(self.convLayer1(x)))
        x=torch.nn.functional.leaky_relu(self.bn2(self.convLayer2(x)))
        
        x=torch.nn.functional.leaky_relu(self.bn3(self.convLayer3(x)))
        x=torch.nn.functional.leaky_relu(self.bn4(self.convLayer4(x)))
        
        x = x.view(x.size(0), -1)
        # x=x.view(64 * 64 * 64, -1) # Flatten layer  x.size(0),   (3 *8)
        
        x=torch.nn.functional.leaky_relu(self.bn5(self.fclayer1(x)))
        
        x=torch.nn.functional.dropout(x,p=.5) 
         
        x=self.fclayer2(x)

        return torch.nn.functional.log_softmax(x, dim=-1) 
    
cnn_model = CNN()
# summary(cnn_model, torch.zeros(1, 3, 32, 32))


In [68]:
# Create the CNN model class
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Define the layers
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=1,
                                      padding=calculate_same_padding(32, 4, 1))
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2,
                                      padding=calculate_same_padding(32, 3, 2))
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1,
                                      padding=calculate_same_padding(32, 3, 1))
        self.bn3 = torch.nn.BatchNorm2d(64)
        self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2,
                                      padding=calculate_same_padding(64, 3, 2))
        self.bn4 = torch.nn.BatchNorm2d(64)
        
        # Calculate shape after convolutions
        test_tensor = torch.zeros(1, 3, 32, 32)
        test_tensor = self.conv1(test_tensor)
        test_tensor = self.conv2(test_tensor)
        test_tensor = self.conv3(test_tensor)
        test_tensor = self.conv4(test_tensor)
        self.flattened_size = test_tensor.shape[1] * test_tensor.shape[2] * test_tensor.shape[3]
        
        # Define dense layers
        self.fc1 = torch.nn.Linear(self.flattened_size, 128)
        self.bn5 = torch.nn.BatchNorm1d(128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x):
        # Apply layers with activation functions
        x = torch.nn.functional.leaky_relu(self.bn1(self.conv1(x)))
        x = torch.nn.functional.leaky_relu(self.bn2(self.conv2(x)))
        x = torch.nn.functional.leaky_relu(self.bn3(self.conv3(x)))
        x = torch.nn.functional.leaky_relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the dense layers
        x = x.view(x.size(0), -1)
        
        x = torch.nn.functional.leaky_relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        
        # Apply softmax activation function
        return torch.nn.functional.softmax(x, dim=-1)

# Initialize the CNN model
cnn_model = CNN()


# Set the Loss Function and Optimizer

In [69]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.0005)

# Write the Train Function

In [ ]:
def train(dataloader, model, criterion, optimizer):
    model.train()
    batch_bar = tqdm(total=len(dataloader), position=0, leave=False, dynamic_ncols=True, desc='Train')
    total_loss = 0
    total_accuracy = 0

    for i, (imgs, labels) in enumerate(dataloader):
        # write your code below for the train function 
        # Forward pass
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0)

        # Update total loss and accuracy
        total_loss += loss.item()
        total_accuracy += correct

        # Update the progress bar
        batch_bar.update()
    batch_bar.close()
    
    return total_loss/len(dataloader), 100*total_accuracy/(len(dataloader)*32)

# Write the Evaluate Function to get the Test Accuracy

In [47]:
def evaluate(dataloader, model):
    model.eval()
    # write your code for the evaluate function
    #raise NotImplemented
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient tracking during evaluation
        for imgs, labels in dataloader:
            # Forward pass
            outputs = model(imgs)
            
            # Compute loss
            loss = criterion(outputs, labels)
            
            # Compute the number of correct predictions
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            
            # Update total loss and total samples
            total_loss += loss.item()
            total_samples += labels.size(0)

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples

    return avg_loss, accuracy

# Train the CNN Model

In [70]:
for epoch in range(0, 1):
    train_loss, train_acc = train(train_dataloader, cnn_model, criterion, optimizer)
    print(f"Epoch {epoch+1}/{10} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_acc:.4f}")

Epoch 1/10 - Train Loss: 1.9901 - Train Accuracy: 49.7541


# Get the Test Accuracy

In [ ]:
test_acc = evaluate(test_dataloader, cnn_model)
# test_accm
test_acc